In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [ ]:
!nvidia-smi

Mon Nov  6 17:00:48 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install -qqq langchain==0.0.266
!pip install PyPDF2==2.12.1
!pip install -qqq sentence_transformers==2.2.2
!pip install -qqq InstructorEmbedding
!pip install -qqq pdf2image
!pip install pgvector psycopg2
!pip install auto_gptq==0.4.2
!pip install -qqq transformers
!pip install -qqq torch
!pip install tensorflow==2.14

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 32.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 49.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.9/134.9 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 21.6 MB/s eta 0:00:00


In [ ]:

import torch
import os
import PyPDF2
from PyPDF2 import PdfFileReader
import pandas as pd
import numpy as np
import json
import psycopg2
import pgvector
from psycopg2.extras import execute_values
from pgvector.psycopg2 import register_vector
from langchain.embeddings import HuggingFaceInstructEmbeddings
from InstructorEmbedding import INSTRUCTOR
from langchain.text_splitter import RecursiveCharacterTextSplitter
from auto_gptq import AutoGPTQForCausalLM
from langchain import HuggingFacePipeline, PromptTemplate
from langchain.chains import RetrievalQA
from transformers import AutoTokenizer, TextStreamer, pipeline


from psycopg2.extras import execute_values
from pgvector.psycopg2 import register_vector
from langchain.vectorstores.pgvector import PGVector
from langchain.document_loaders import TextLoader
from langchain.docstore.document import Document

DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"

/usr/local/lib/python3.10/dist-packages/InstructorEmbedding/instructor.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange


In [ ]:
# Function to extract text from a PDF file
def extract_text_from_pdf(pdf_path):
    text = ""
    with open(pdf_path, 'rb') as pdf_file:
        pdf_reader = PyPDF2.PdfFileReader(pdf_file,strict=False)
        for page in range(pdf_reader.numPages):
            text += pdf_reader.getPage(page).extractText()
    return text

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Physician AI/Primary Care (Adult)/df1.csv")
df.head()

,Unnamed: 0,ID,Folder_Name,Sub_Folder_Name,PDF_Name,Chunk_Number,PDF_Text,Embeddings
0,0,1,//content//drive//MyDrive//Primary Care (Adult...,Patient perspective,Patient perspective Chronic urticaria.pdf,0,"9//25//23, 8:55 PM Patient perspective: Chroni...","[-0.0023331299889832735, -0.006728040520101786..."
1,1,2,//content//drive//MyDrive//Primary Care (Adult...,Patient perspective,Patient perspective Chronic urticaria.pdf,1,provider's examination and assessment of a pat...,"[-0.01195523701608181, -0.012462286278605461, ..."
2,2,3,//content//drive//MyDrive//Primary Care (Adult...,Patient perspective,Patient perspective Chronic urticaria.pdf,2,"and natural history"".)\n(See ""Chronic spontane...","[-0.017680862918496132, -0.0015703006647527218..."
3,3,4,//content//drive//MyDrive//Primary Care (Adult...,Patient perspective,Patient perspective Chronic urticaria.pdf,3,though I'd taken this medication in the past. ...,"[-0.006123702973127365, -0.007063134573400021,..."
4,4,5,//content//drive//MyDrive//Primary Care (Adult...,Patient perspective,Patient perspective Chronic urticaria.pdf,4,"overwhelming, and frustrating to continue to s...","[-0.014102621003985405, -0.015657760202884674,..."


In [ ]:

df=df.drop(['Unnamed: 0','Embeddings'],axis=1)

In [ ]:
df=df.head(1000)

In [ ]:
df.shape

(1000, 6)

In [ ]:
df.head()

,ID,Folder_Name,Sub_Folder_Name,PDF_Name,Chunk_Number,PDF_Text
0,1,//content//drive//MyDrive//Primary Care (Adult...,Patient perspective,Patient perspective Chronic urticaria.pdf,0,"9//25//23, 8:55 PM Patient perspective: Chroni..."
1,2,//content//drive//MyDrive//Primary Care (Adult...,Patient perspective,Patient perspective Chronic urticaria.pdf,1,provider's examination and assessment of a pat...
2,3,//content//drive//MyDrive//Primary Care (Adult...,Patient perspective,Patient perspective Chronic urticaria.pdf,2,"and natural history"".)\n(See ""Chronic spontane..."
3,4,//content//drive//MyDrive//Primary Care (Adult...,Patient perspective,Patient perspective Chronic urticaria.pdf,3,though I'd taken this medication in the past. ...
4,5,//content//drive//MyDrive//Primary Care (Adult...,Patient perspective,Patient perspective Chronic urticaria.pdf,4,"overwhelming, and frustrating to continue to s..."


In [ ]:
#load documents from Pandas dataframe for insertion into database
from langchain.document_loaders import DataFrameLoader

loader = DataFrameLoader(df, page_content_column = 'PDF_Text')
docs = loader.load()

In [ ]:
embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-large", model_kwargs={"device": DEVICE})

load INSTRUCTOR_Transformer
max_seq_length  512


In [ ]:
CONNECTION_STRING = "*************"

In [ ]:
# Create a PGVector instance to house the documents and embeddings
from langchain.vectorstores.pgvector import DistanceStrategy
from langchain.vectorstores.pgvector import PGVector

In [ ]:
# create vector store at postgres
db = PGVector.from_documents(
    documents= docs,
    embedding = embeddings,
    collection_name= "sample_df", #this should be changed when creating another vector store with different chunk size and overlap
    distance_strategy = DistanceStrategy.COSINE,
    connection_string= CONNECTION_STRING)

In [ ]:
model_name_or_path = "TheBloke/Llama-2-13B-chat-GPTQ"
model_basename = "model"


tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

model = AutoGPTQForCausalLM.from_quantized(
    model_name_or_path,
    revision="gptq-4bit-128g-actorder_True",
    model_basename=model_basename,
    use_safetensors=True,
    trust_remote_code=True,
    inject_fused_attention=False,
    device=DEVICE,
    quantize_config=None,
)

In [ ]:
DEFAULT_SYSTEM_PROMPT = """
You are a helpful, respectful and honest assistant. give answer for any questions.
 """.strip()


def generate_prompt(prompt: str, system_prompt: str = DEFAULT_SYSTEM_PROMPT) -> str:
    return f"""
[INST] <<SYS>>
{system_prompt}
<</SYS>>

{prompt} [/INST]
""".strip()

In [ ]:
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

In [ ]:
text_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=4096,
    temperature=0,
    top_p=0.95,
    repetition_penalty=1.15,
    streamer=streamer,
)

The model 'LlamaGPTQForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PersimmonForCausalLM', 'PLBartForCausalLM', 'Prophe

In [ ]:
llm = HuggingFacePipeline(pipeline=text_pipeline, model_kwargs={"temperature": 0})

In [ ]:
SYSTEM_PROMPT = "use the following system pieces of context to answer the questions. If you don't know the answer, just say you don't know, don't try to make up an answer."

template = generate_prompt("""
{context}

Question: {question}
""",
    system_prompt=SYSTEM_PROMPT,
)

In [ ]:
#SYSTEM_PROMPT = "give answer from using external data or internal provided context"

#template = generate_prompt("""
#{context}

#Question: {question}
#""",
    #system_prompt=SYSTEM_PROMPT,)

In [ ]:
prompt = PromptTemplate(template=template, input_variables=["context", "question"])

In [ ]:
rag_pipeline = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={"k": 2}),
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt},
)

In [ ]:
query1 = rag_pipeline("what is the main blood groups in human body and who is the universal?")

 Based on the information provided by UpToDate, there are several main blood groups in the human body, including:

1. ABO blood group system: This system includes four main blood types - A, B, AB, and O - which are determined by the presence or absence of specific antigens on the surface of red blood cells.
2. Rh blood group system: This system includes two main blood types - positive and negative - which are determined by the presence or absence of a specific antigen on the surface of red blood cells.
3. Kell blood group system: This system includes several subtypes, but the most important one is the Kell antigen, which is found on the surface of red blood cells and can cause hemolytic transfusion reactions.
4. Duffy blood group system: This system includes two main blood types - Fy(a-b-) and Fy(a+b+), which are determined by the presence or absence of specific antigens on the surface of red blood cells.

There is no such thing as a "universal" blood type, as each person has a unique 